# RiemannianLayoutNet 수학 원리 정리

## 1. 특징 추출 (Feature Extraction)

### 1.1. 텍스트 밀도 (Density)
- 이진화 후 적분 영상을 이용해 지역별 텍스트 밀도 계산
- 수식:
  $$
  D(i,j) = \frac{1}{w^2 \cdot 255} \sum_{u=i-\frac{w}{2}}^{i+\frac{w}{2}} \sum_{v=j-\frac{w}{2}}^{j+\frac{w}{2}} B(u,v)
  $$
- GaussianBlur로 스무딩하여 노이즈 완화

### 1.2 수직 그래디언트 (Vertical Gradient)
- 픽셀 밝기 변화의 수직 방향 그래디언트 강조
- 수식:
  $$
  G_y(i,j) = \text{GaussianBlur}\bigl(|\partial_y I(i,j)|\bigr)
  $$
- 최댓값으로 정규화: $G_y \leftarrow G_y / \max(G_y)$

## 2. 리만 메트릭 텐서 (Riemannian Metric Tensor)
- 각 픽셀 위치마다 2×2 대각 행렬로 정의:
  $$
  g(i,j) = \begin{pmatrix}
    w(i,j)\,\gamma & 0 \\
    0 & w(i,j)
  \end{pmatrix},
  $$
- 경계 강조 가중치:
  $$
  w(i,j) = 1 + \alpha\, \frac{\min(\|\nabla D(i,j)\|^2,\tau)}{\tau},
  $$
  - $\alpha$: 메트릭 가중치 계수
  - $\tau$: 그래디언트 임계값
  - $\gamma$: 수직 강조 계수
- 리만 그래디언트 업데이트에 사용:
  $$
  \Delta x = G^{-1} \nabla E = \begin{pmatrix} g^{11} & 0 \\ 0 & g^{22} \end{pmatrix}
    \begin{pmatrix} \partial_y E \\ \partial_x E \end{pmatrix}
  $$

## 3. 액티브 컨투어 최적화 (Active Contour Optimization)
- **내부 에너지 (Internal Energy)**:
  $$
  E_{int} = \alpha \|x_{i+1}-x_i\|^2 + \beta \|x_{i+1}-2x_i+x_{i-1}\|^2
  $$
- **외부 에너지 (External Energy)**:
  $$
  E_{ext} = -\gamma \, \nabla I(x_i)
  $$
- **업데이트 방정식**:
  $$
  x_i \leftarrow x_i - G^{-1}(\nabla E_{int} + \nabla E_{ext}).
  $$


In [ ]:
# RiemannianDocSegNet: 파이썬에 최적화된 리만 기하학 기반 문서 분할
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
import warnings
warnings.filterwarnings('ignore')

class SyntheticDocumentGenerator:
    """테스트용 합성 문서 생성기"""
    def __init__(self):
        pass
    def generate_document(self, doc_type="text", width=800, height=1100):
        """테스트용 합성 문서 생성"""
        # 빈 문서 생성
        img = np.ones((height, width, 3), dtype=np.uint8) * 255

        if doc_type == "text":
            # 헤더 섹션
            cv2.rectangle(img, (50, 50), (width-50, 150), (240, 240, 240), -1)
            for y in range(70, 140, 30):
                length = np.random.randint(100, width-150)
                cv2.rectangle(img, (80, y), (80 + length, y + 10), (200, 200, 200), -1)

            # 중간 섹션 (단락 포함)
            for i in range(4):
                y_start = 200 + i * 220
                # 단락 헤더
                cv2.rectangle(img, (80, y_start), (width-80, y_start + 20), (220, 220, 220), -1)

                # 단락 텍스트
                for j in range(6):
                    y_pos = y_start + 40 + j * 25
                    length = np.random.randint(width-200, width-100)
                    cv2.rectangle(img, (80, y_pos), (80 + length, y_pos + 10), (230, 230, 230), -1)

            # 푸터 섹션
            cv2.rectangle(img, (50, height-120), (width-50, height-50), (240, 240, 240), -1)
            cv2.rectangle(img, (100, height-100), (300, height-80), (220, 220, 220), -1)

        elif doc_type == "form":
            # 폼 형태 문서
            # 제목
            cv2.rectangle(img, (width//4, 50), (3*width//4, 100), (240, 240, 240), -1)

            # 폼 필드
            for i in range(8):
                y_pos = 150 + i * 100
                # 라벨
                cv2.rectangle(img, (80, y_pos), (width//3, y_pos + 20), (230, 230, 230), -1)
                # 입력 필드
                cv2.rectangle(img, (width//3 + 20, y_pos), (width-80, y_pos + 40), (245, 245, 245), 2)

            # 서명 영역
            cv2.rectangle(img, (80, height-150), (width//2, height-80), (240, 240, 240), 2)

        elif doc_type == "table":
            # 표 형태 문서
            # 헤더
            cv2.rectangle(img, (50, 50), (width-50, 120), (240, 240, 240), -1)

            # 표 헤더
            cv2.rectangle(img, (100, 200), (width-100, 250), (220, 220, 220), -1)

            # 표 행
            for i in range(12):
                y_pos = 250 + i * 40
                cv2.rectangle(img, (100, y_pos), (width-100, y_pos + 40), (240, 240, 240), 1)

                # 표 셀
                for j in range(3):
                    x_pos = 100 + j * (width-200)//3
                    cv2.line(img, (x_pos, y_pos), (x_pos, y_pos + 40), (220, 220, 220), 1)
                    # 셀 내용
                    content_width = np.random.randint(30, (width-200)//3 - 30)
                    cv2.rectangle(img, (x_pos + 15, y_pos + 15), (x_pos + 15 + content_width, y_pos + 25), (230, 230, 230), -1)
            # 푸터
            cv2.rectangle(img, (50, height-100), (width-50, height-50), (240, 240, 240), -1)

        # 시각적 노이즈/텍스처 추가
        noise = np.random.normal(0, 2, img.shape).astype(np.int8)
        img = np.clip(img + noise, 0, 255).astype(np.uint8)

        return img

    def create_test_documents(self, num_docs=3):
        """여러 테스트 문서 생성"""
        doc_types = ["text", "form", "table"]
        filenames = []

        for i in range(num_docs):
            doc_type = doc_types[i % len(doc_types)]
            filename = f"synthetic_document_{i+1}_{doc_type}.jpg"
            img = self.generate_document(doc_type)
            cv2.imwrite(filename, img)
            filenames.append(filename)
            print(f"합성 문서 생성 완료: {filename}")

        return filenames


class RLFeatureExtractor:
    """리만 특징 추출기"""

    def __init__(self):
        self.feature_types = ['density', 'gradient']
        print("빠른 리만 특징 추출기 초기화")

    def extract(self, image):
        """이미지에서 특징 추출 - 파이썬 최적화 버전"""
        # 그레이스케일 변환
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image

        # 크기 정규화 (메모리 효율성)
        max_dim = 1000
        h, w = gray.shape
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            new_h, new_w = int(h * scale), int(w * scale)
            gray = cv2.resize(gray, (new_w, new_h))
        # 노이즈 제거 (빠른 블러)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        # 특징 배열 초기화
        features = np.zeros((gray.shape[0], gray.shape[1], len(self.feature_types)))
        # 1. 빠른 텍스트 밀도 특징 추출
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        # 적분 이미지를 사용한 효율적인 계산
        window_size = 15  # 단일 윈도우 크기 유지 (다중 스케일보다 빠름)
        density = np.zeros_like(gray, dtype=np.float32)
        integral = cv2.integral(binary)
        # 벡터화된 윈도우 인덱스 생성
        h, w = gray.shape
        y_indices, x_indices = np.meshgrid(np.arange(window_size//2, h - window_size//2), np.arange(window_size//2, w - window_size//2), indexing='ij')
        y1, x1 = y_indices - window_size//2, x_indices - window_size//2
        y2, x2 = y_indices + window_size//2 + 1, x_indices + window_size//2 + 1
        # 적분 이미지에서 윈도우 합 추출
        window_sums = (integral[y2, x2] - integral[y2, x1] - integral[y1, x2] + integral[y1, x1])
        # 윈도우 영역과 최대 픽셀값으로 정규화
        density[window_size//2:h - window_size//2, window_size//2:w - window_size//2] = \
            window_sums / (window_size * window_size * 255)
        # 더 효율적인 밀도맵 스무딩
        density = cv2.GaussianBlur(density, (window_size, window_size), 0)
        # 2. 수직 그래디언트 특징 추출 - 수직 변화 강조
        # NumPy 그래디언트 사용 (OpenCV 호환성 문제 해결)
        grad_y, _ = np.gradient(gray.astype(np.float32) / 255.0)
        # 수직 변화 강조 (문서 분할에 중요)
        v_gradient = cv2.GaussianBlur(np.abs(grad_y), (3, 11), 0)
        if v_gradient.max() > 0:
            v_gradient = v_gradient / v_gradient.max()
        # 특징 저장
        features[:,:,0] = density
        features[:,:,1] = v_gradient
        print(f"{len(self.feature_types)}개 특징 추출 완료")
        return features

class RLMetricComputer:
    """리만 메트릭 계산기"""

    def __init__(self):
        self.alpha = 10.0  # 메트릭 가중치 계수
        print("리만 메트릭 계산기 초기화")

    def compute_metric(self, features):
        """특징에서 리만 메트릭 텐서 계산"""
        h, w, _= features.shape
        # 메트릭 텐서 초기화
        metric_tensor = np.zeros((h, w, 2, 2), dtype=np.float32)
        # 주요 특징으로 텍스트 밀도 사용
        main_feature = features[:,:,0]
        print("리만 메트릭 텐서 계산 중...")
        # NumPy 그래디언트 사용 (OpenCV 호환성 문제 해결)
        grad_y, grad_x = np.gradient(main_feature)
        gradient_magnitude = grad_x**2 + grad_y**2
        # 경계 가중치 계산 - 비선형성 유지하면서 속도 최적화
        threshold = 0.2 * gradient_magnitude.max() if gradient_magnitude.max() > 0 else 0.0001
        boundary_weight = 1.0 + self.alpha * np.minimum(gradient_magnitude, threshold) / (threshold + 1e-8)
        # 방향성 가중치 적용
        vertical_emphasis = 1.5
        # 메트릭 텐서 구성
        metric_tensor[:,:,0,0] = boundary_weight * vertical_emphasis  # g_11 (수직)
        metric_tensor[:,:,1,1] = boundary_weight  # g_22 (수평)
        # 비대각 요소 (최적화를 위해 0으로 설정)
        metric_tensor[:,:,0,1] = 0
        metric_tensor[:,:,1,0] = 0
        print("메트릭 텐서 계산 완료")
        return metric_tensor


class RLActiveContour:
    """리만 액티브 컨투어 모델"""
    def __init__(self, metric_tensor, alpha=0.1, beta=0.5, gamma=0.8):
        self.metric_tensor = metric_tensor
        self.height, self.width = metric_tensor.shape[0:2]
        self.alpha = alpha  # 탄성 가중치
        self.beta = beta    # 강성 가중치
        self.gamma = gamma  # 외부 힘 가중치
        print("빠른 리만 액티브 컨투어 모델 초기화")

    def optimize_boundary(self, initial_contour, feature_map, max_iterations=50, tolerance=0.5):
        """경계선 최적화 - 파이썬 최적화 버전"""
        print("경계선 최적화 시작...")
        contour = initial_contour.copy()
        n_points = contour.shape[0]
        # 점 분포 정규화
        contour = self._resample_contour(contour, n_points)
        # NumPy 그래디언트 사용 (OpenCV 호환성 문제 해결)
        gy, gx = np.gradient(feature_map)
        # 최적화 반복
        for iteration in range(max_iterations):
            # 수렴 확인을 위해 이전 윤곽선 저장
            prev_contour = contour.copy()
            # 내부 힘 계산 (벡터화)
            internal_force = self._compute_internal_force_fast(contour)
            # 외부 힘 계산 (벡터화)
            external_force = self._compute_external_force_fast(contour, gy, gx)
            # 가중치와 함께 힘 결합
            update = self.alpha * internal_force + self.gamma * external_force
            # 리만 메트릭을 사용한 윤곽선 업데이트 - 벡터화
            y_indices = np.clip(np.round(contour[:,0]).astype(int), 0, self.height-1)
            x_indices = np.clip(np.round(contour[:,1]).astype(int), 0, self.width-1)
            # 각 점에 대해 리만 그래디언트 계산
            for i in range(n_points):
                y, x = y_indices[i], x_indices[i]
                metric = self.metric_tensor[y, x]
                # 2x2 역행렬 직접 계산 (빠른 수학적 계산)
                a, b = metric[0, 0], metric[0, 1]
                c, d = metric[1, 0], metric[1, 1]
                det = a*d - b*c
                if abs(det) < 1e-8:
                    # 특이 행렬인 경우 단순 업데이트
                    contour[i] += 0.5 * update[i]
                else:
                    # 리만 그래디언트 계산
                    inv_det = 1.0 / det
                    inv_metric = np.array([
                        [d*inv_det, -b*inv_det],
                        [-c*inv_det, a*inv_det]
                    ])
                    riemannian_gradient = inv_metric @ update[i]
                    # 안정성을 위한 업데이트 크기 제한
                    update_norm = np.sqrt(np.sum(riemannian_gradient**2))
                    if update_norm > 2.0:
                        riemannian_gradient = 2.0 * riemannian_gradient / (update_norm + 1e-8)
                    # 윤곽선 점 업데이트
                    contour[i] += riemannian_gradient
            # 이미지 경계 내에 윤곽선 유지
            contour[:,0] = np.clip(contour[:,0], 0, self.height-1)
            contour[:,1] = np.clip(contour[:,1], 0, self.width-1)
            # 균일한 점 분포를 위한 윤곽선 리샘플링
            contour = self._resample_contour(contour, n_points)
            # 수렴 확인
            change = np.mean(np.sqrt(np.sum((contour - prev_contour)**2, axis=1)))
            if iteration % 10 == 0:
                print(f"반복 {iteration}, 변화량: {change:.6f}")
            if change < tolerance:
                print(f"{iteration}회 반복 후 수렴")
                break
        print("경계선 최적화 완료")
        return contour

    def _compute_internal_force_fast(self, contour):
        """내부 힘 벡터화 계산 - 최적화 버전"""
        # 인접 점 계산 (빠른 벡터화)
        next_points = np.roll(contour, -1, axis=0)
        prev_points = np.roll(contour, 1, axis=0)
        # 탄성력 + 강성력 한번에 계산
        internal_force = self.alpha * (next_points + prev_points - 2 * contour)
        internal_force += self.beta * (next_points + prev_points - 2 * contour)
        return internal_force

    def _compute_external_force_fast(self, contour, gradient_y, gradient_x):
        """외부 힘 벡터화 계산 - 최적화 버전"""
        external_force = np.zeros_like(contour, dtype=np.float32)
        # 각 점의 인덱스 계산
        y_indices = np.clip(np.round(contour[:,0]).astype(int), 0, self.height-1)
        x_indices = np.clip(np.round(contour[:,1]).astype(int), 0, self.width-1)
        # 그래디언트 기반 힘 - 벡터화된 인덱싱
        external_force[:,0] = -gradient_y[y_indices, x_indices]
        external_force[:,1] = -gradient_x[y_indices, x_indices]
        # 경계 제약 조건 추가
        margin = 5
        # 빠른 마스킹 인덱싱
        external_force[:,0] += np.where(y_indices <= margin, (margin - y_indices) * 0.5, 0)
        external_force[:,0] -= np.where(y_indices >= self.height - margin, (y_indices - (self.height - margin)) * 0.5, 0)
        external_force[:,1] += np.where(x_indices <= margin, (margin - x_indices) * 0.5, 0)
        external_force[:,1] -= np.where(x_indices >= self.width - margin, (x_indices - (self.width - margin)) * 0.5, 0)
        return external_force

    def _resample_contour(self, contour, n_points):
        """균일한 점 분포를 위한 윤곽선 리샘플링"""
        # 최소 윤곽선 크기 확인
        n = len(contour)
        if n < 3:
            return contour
        # 윤곽선을 따라 누적 거리 계산
        cumulative_dist = np.zeros(n+1)
        for i in range(1, n+1):
            prev_idx = (i-1) % n
            curr_idx = (i) % n
            segment_length = np.sqrt(np.sum((contour[curr_idx] - contour[prev_idx])**2))
            cumulative_dist[i] = cumulative_dist[i-1] + segment_length
        total_length = cumulative_dist[-1]
        if total_length < 1e-8:
            return contour
        # 균등한 거리에서 리샘플링
        new_contour = np.zeros((n_points, 2), dtype=np.float32)
        for i in range(n_points):
            target_dist = i * total_length / n_points
            # 목표 거리를 포함하는 세그먼트 찾기
            idx = np.searchsorted(cumulative_dist, target_dist) - 1
            idx = max(0, min(idx, n-1))
            next_idx = (idx + 1) % n
            segment_length = cumulative_dist[idx+1] - cumulative_dist[idx]
            if segment_length > 0:
                alpha = (target_dist - cumulative_dist[idx]) / segment_length
            else:
                alpha = 0
            # 선형 보간
            new_contour[i] = (1-alpha) * contour[idx] + alpha * contour[next_idx]
        return new_contour


class RLSegmentor:
    def __init__(self):
        self.feature_extractor = RLFeatureExtractor()
        self.metric_computer = RLMetricComputer()
        print("빠른 리만 문서 분할기 초기화")

    def load_document(self, path):
        """문서 이미지 로드"""
        try:
            print(f"문서 로드 중: {path}")
            image = cv2.imread(path)

            if image is None:
                raise ValueError(f"이미지 로드 실패: {path}")

            # BGR에서 RGB로 변환
            if len(image.shape) == 3:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            return image

        except Exception as e:
            print(f"문서 로드 오류: {e}")
            # 오류 시 빈 이미지 반환
            return np.ones((1000, 800, 3), dtype=np.uint8) * 255

    def segment(self, document_path, num_segments=3, visualize=True):
        """문서 분할 수행 - 파이썬 최적화 버전"""
        start_time = time.time()

        # 1. 문서 이미지 로드
        document_image = self.load_document(document_path)
        print(f"문서 크기: {document_image.shape}")

        # 그레이스케일 변환
        if len(document_image.shape) == 3:
            gray = cv2.cvtColor(document_image, cv2.COLOR_RGB2GRAY)
        else:
            gray = document_image
        # 2. 특징 추출
        features = self.feature_extractor.extract(gray)
        # 3. 리만 메트릭 계산
        metric_tensor = self.metric_computer.compute_metric(features)
        # 4. 초기 경계선 설정
        height, width = gray.shape
        boundary_lines = []
        for i in range(1, num_segments):
            # 수평 분할선
            y_pos = int(i * height / num_segments)
            # 더 효율적인 초기 컨투어 - 너무 많은 점은 비효율적
            step_size = max(10, width//50)  # 더 적은 수의 초기 점
            initial_contour = np.array([[y_pos, x] for x in range(0, width, step_size)])
            boundary_lines.append(initial_contour)
        # 5. 리만 액티브 컨투어를 사용한 경계선 최적화
        optimized_boundaries = []
        active_contour = RLActiveContour(metric_tensor)
        for i, initial_contour in enumerate(boundary_lines):
            print(f"경계선 {i+1}/{len(boundary_lines)} 최적화 중...")
            # 밀도 특징 맵 사용
            feature_map = features[:,:,0]
            optimal_boundary = active_contour.optimize_boundary(
                initial_contour, feature_map, max_iterations=50
            )
            optimized_boundaries.append(optimal_boundary)
        # 6. 문서 세그먼트 추출 (최적화된 버전)
        segments = self._extract_segments_fast(gray, optimized_boundaries)
        elapsed_time = time.time() - start_time
        print(f"총 처리 시간: {elapsed_time:.2f}초")
        # 7. 결과 시각화
        if visualize:
            self._visualize_results(document_image, features, optimized_boundaries, segments)
        return segments, optimized_boundaries

    def _extract_segments_fast(self, image, boundaries):
        """경계선 기반 문서 세그먼트 추출 - 최적화 버전"""
        height, width = image.shape
        segments = []
        # 경계 좌표를 정수로 반올림
        int_boundaries = []
        for boundary in boundaries:
            int_boundary = np.round(boundary).astype(int)
            int_boundary[:,0] = np.clip(int_boundary[:,0], 0, height-1)
            int_boundary[:,1] = np.clip(int_boundary[:,1], 0, width-1)
            int_boundaries.append(int_boundary)

        # 세그먼트 마스크 생성
        segment_mask = np.zeros((height, width), dtype=np.int32)

        # 상단 세그먼트 추출
        if len(int_boundaries) > 0:
            top_mask = np.zeros((height, width), dtype=np.uint8)
            # 첫 번째 경계선 위의 영역
            pts = np.vstack([
                [[0, 0]],
                [[width-1, 0]],
                [[width-1, int_boundaries[0][0, 0]]],
                *[[x, y] for y, x in int_boundaries[0][::-1]],
                [[0, int_boundaries[0][-1, 0]]]
            ]).astype(np.int32)
            cv2.fillPoly(top_mask, [pts], 1)
            segment_mask[top_mask == 1] = 1
            # 세그먼트 추출 (빠른 마스킹)
            segment_img = np.zeros_like(image)
            segment_img[top_mask == 1] = image[top_mask == 1]
            segments.append(segment_img)

        # 중간 세그먼트 추출
        for i in range(len(int_boundaries) - 1):
            mid_mask = np.zeros((height, width), dtype=np.uint8)
            # 두 경계선 사이 영역
            pts = np.vstack([
                *[[x, y] for y, x in int_boundaries[i]],
                *[[x, y] for y, x in int_boundaries[i+1][::-1]]
            ]).astype(np.int32)
            cv2.fillPoly(mid_mask, [pts], 1)
            segment_mask[mid_mask == 1] = i + 2

            # 세그먼트 추출
            segment_img = np.zeros_like(image)
            segment_img[mid_mask == 1] = image[mid_mask == 1]
            segments.append(segment_img)

        # 하단 세그먼트 추출
        if len(int_boundaries) > 0:
            bottom_mask = np.zeros((height, width), dtype=np.uint8)
            # 마지막 경계선 아래 영역
            pts = np.vstack([
                *[[x, y] for y, x in int_boundaries[-1]],
                [[width-1, int_boundaries[-1][-1, 0]]],
                [[width-1, height-1]],
                [[0, height-1]],
                [[0, int_boundaries[-1][0, 0]]]
            ]).astype(np.int32)
            cv2.fillPoly(bottom_mask, [pts], 1)
            segment_mask[bottom_mask == 1] = len(int_boundaries) + 1
            # 세그먼트 추출
            segment_img = np.zeros_like(image)
            segment_img[bottom_mask == 1] = image[bottom_mask == 1]
            segments.append(segment_img)

        return segments

    def _visualize_results(self, image, features, boundaries, segments):
        """분할 결과 시각화"""
        # 시각화를 위한 그림 생성
        plt.figure(figsize=(20, 15))

        # 1. 원본 이미지
        plt.subplot(2, 2, 1)
        plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        plt.title('원본 문서', fontsize=15)
        plt.axis('off')

        # 2. 특징 맵
        plt.subplot(2, 2, 2)
        plt.imshow(features[:,:,0], cmap='jet')
        plt.title('텍스트 밀도 특징 맵', fontsize=15)
        plt.colorbar()
        plt.axis('off')

        # 3. 경계선이 표시된 이미지
        plt.subplot(2, 2, 3)
        plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        for boundary in boundaries:
            plt.plot(boundary[:,1], boundary[:,0], 'r-', linewidth=2)
        plt.title('최적화된 세그먼트 경계선', fontsize=15)
        plt.axis('off')

        # 4. 분할 시각화
        plt.subplot(2, 2, 4)
        colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta']
        overlay = np.zeros((*image.shape[:2], 3), dtype=np.uint8)

        if len(image.shape) == 3:
            grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            grayscale = image

        # 각 세그먼트에 색상 마스크 적용
        for i, segment in enumerate(segments):
            color_idx = i % len(colors)
            color_val = np.array(plt.cm.colors.to_rgb(colors[color_idx])) * 255
            mask = segment > 0
            for c in range(3):
                overlay[:,:,c][mask] = color_val[c]

        # 원본 이미지와 오버레이 블렌딩
        alpha = 0.5
        blended = cv2.addWeighted(
            cv2.cvtColor(grayscale, cv2.COLOR_GRAY2RGB),
            1 - alpha,
            overlay,
            alpha,
            0
        )
        plt.imshow(blended)
        plt.title('분할 결과', fontsize=15)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig('document_segmentation_result.png', dpi=300, bbox_inches='tight')
        plt.show()
        # 개별 세그먼트 시각화
        plt.figure(figsize=(15, 5 * ((len(segments) + 2) // 3)))
        for i, segment in enumerate(segments):
            plt.subplot(((len(segments) + 2) // 3), 3, i+1)
            plt.imshow(segment, cmap='gray')
            plt.title(f'세그먼트 {i+1}', fontsize=12)
            plt.axis('off')
        plt.tight_layout()
        plt.savefig('individual_segments.png', dpi=300, bbox_inches='tight')
        plt.show()

# 메인 진입점
def main():
    # 합성 문서 생성
    document_generator = SyntheticDocumentGenerator()
    test_files = document_generator.create_test_documents(3)
    # 분할기 초기화
    segmenter = RLSegmentor()
    # 각 문서 처리
    for i, file_path in enumerate(test_files):
        print(f"\n\n===== 문서 {i+1} 처리 중 =====")
        segments, boundaries = segmenter.segment(file_path, num_segments=3)
        print(f"문서 {i+1} 처리 완료: {len(segments)}개 세그먼트 추출됨")
    print("\n모든 문서 처리 완료")

# 실행
if __name__ == "__main__":
    main()

: 